In [1]:
import pandas as pd

%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train=pd.read_csv('train.csv',parse_dates = ["datetime"])
print(train.shape)
train.head()

(10886, 12)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [3]:
train["datetime"].dt.year
test = pd.read_csv('test.csv', parse_dates = ["datetime"])

print(test.shape)
test.head()

(6493, 9)


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


## Train

In [4]:
#season -> holiday workingday weather temp atemp humidity windspe

feature_names = ["season","holiday","workingday","weather","temp","atemp", "humidity","windspeed"]

X_train = train[feature_names]

print(X_train.shape)
X_train.head()

(10886, 8)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1,0,0,1,9.84,14.395,81,0.0
1,1,0,0,1,9.02,13.635,80,0.0
2,1,0,0,1,9.02,13.635,80,0.0
3,1,0,0,1,9.84,14.395,75,0.0
4,1,0,0,1,9.84,14.395,75,0.0


In [5]:
X_test = test[feature_names]

print(X_test.shape)
X_test.head()

(6493, 8)


,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,1,0,1,1,10.66,11.365,56,26.0027
1,1,0,1,1,10.66,13.635,56,0.0000
2,1,0,1,1,10.66,13.635,56,0.0000
3,1,0,1,1,10.66,12.880,56,11.0014
4,1,0,1,1,10.66,12.880,56,11.0014


In [6]:
label_name = "count"
label = ["casual","registered","count"]
y_train = train[label_name]

y_train.head()

0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

In [7]:
# from sklearn.tree import DecisionTreeRegressor

# model = DecisionTreeRegressor(random_state=31)
# model

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(random_state=31)
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=31,
           verbose=0, warm_start=False)

## Score

In [8]:
import numpy as np
from sklearn.metrics import make_scorer

def rmsle(predict, actual):
    predict = np.array(predict)
    actual = np.array(actual)
    
    log_predict = np.log(predict +1)
    log_actual = np.log(actual +1)
    
    difference = log_predict - log_actual
    difference = np.square(difference)
    
    mean_difference = difference.mean()
    
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [9]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=31,
           verbose=0, warm_start=False)

In [10]:
predictions = model.predict(X_test)

print(predictions.shape)
predictions

(6493,)


array([ 234.3       ,   71.9       ,   71.9       , ...,   92.23333333,
         53.93333333,   23.13333333])

In [11]:
from sklearn.cross_validation import cross_val_score

score = cross_val_score(model, X_train, y_train, cv = 20, scoring=rmsle_scorer).mean()

# score = -1.0 * score

print("Score = {0:.5f}".format(score))

/Users/Philip/.pyenv/versions/3.5.2/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Score = 1.38983


## Submit

In [12]:
submission = pd.read_csv("sampleSubmission.csv")
submission["count"]= predictions

print(submission.shape)
submission.head()

(6493, 2)


,datetime,count
0,2011-01-20 00:00:00,234.3
1,2011-01-20 01:00:00,71.9
2,2011-01-20 02:00:00,71.9
3,2011-01-20 03:00:00,113.5
4,2011-01-20 04:00:00,113.5


In [14]:
submission.to_csv("baseline-script.csv", index=False)